# Export evaluation results from Urban-PLUMBER
- This script is used to export evaluation results of single-point simulations from Urban-PLUMBER.

In [1]:
import pandas as pd
import numpy as np
from io import StringIO

In [4]:
model_list = ['CLMU5', 'NOAH-SLUCM']
site_list = ['AU-Preston', 'AU-SurreyHills', 'CA-Sunset', 'FI-Kumpula', 'FI-Torni',
             'FR-Capitole', 'GR-HECKOR', 'JP-Yoyogi', 'KR-Jungnang', 'KR-Ochang',
             'MX-Escandon', 'NL-Amsterdam', 'PL-Lipowa', 'PL-Narutowicza', 'SG-TelokKurau',
             'UK-KingsCollege', 'UK-Swindon', 'US-Baltimore', 'US-Minneapolis1', 'US-Minneapolis2', 'US-WestPhoenix']
md_header = "| flux | experiment | MAE | MBE | nSD | R | 5th | 95th | RMSE | cRMSE | AMBE | 1-nSD | 1-R | nSkewness | nKurtosis | Overlap |\n"             

In [72]:
df_list = []
for model in model_list:
    for site in site_list:
    #for site in ['AU-SurreyHills']:
        filename = f"./download/{site}-{model}.md"
        with open(filename, "r", encoding="utf-8") as f:
            lines = f.readlines()
        #table_lines = [line for line in lines if '|' in line and '-' not in line]
        table_lines = [line for line in lines if '|' in line]
        # Prepend the header
        #full_table_lines = [md_header] + table_lines
        # Optional: Clean whitespace
        #table_text = ''.join(full_table_lines).strip()
        table_text = ''.join(table_lines).strip()
        # Step 3: Convert to CSV-like format
        table_io = StringIO(table_text)
        df = pd.read_csv(table_io, sep='|', engine='python', skiprows=[1])

        # Step 4: Drop any unwanted blank columns from markdown formatting
        df = df.drop(columns=[col for col in df.columns if col.strip() == ''])
        df = df.loc[:, ~df.columns.str.startswith('Unnamed:')]
        # Step 5: Strip whitespace from column names
        df.columns = df.columns.str.strip()
        df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
        df['site'] = site
        df['model'] = model
        # Step 6: Convert numeric columns (all except 'flux', 'experiment', 'site', 'model')
        non_numeric_cols = ['flux', 'experiment', 'site', 'model']
        numeric_cols = [col for col in df.columns if col not in non_numeric_cols]
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
        df_list.append(df)
df_site_model = pd.concat(df_list, ignore_index=True)
df_site_model.to_csv("./data_for_figure/site_model_summary.csv", index=False)    
df_site_model.head() 

,flux,experiment,MAE,MBE,nSD,R,5th,95th,RMSE,cRMSE,AMBE,1-nSD,1-R,nSkewness,nKurtosis,Overlap,site,model
0,SWup,baseline,2.98882,0.900153,1.005940,0.996055,0.478753,0.303096,4.25611,0.089283,0.900153,0.005936,0.003945,0.086279,0.178836,0.068732,AU-Preston,CLMU5
1,SWup,detailed,2.73408,-0.179863,1.013290,0.996805,0.545667,2.264600,3.80392,0.081552,0.179863,0.013288,0.003195,0.026835,0.051883,0.063041,AU-Preston,CLMU5
2,LWup,baseline,12.16520,9.981450,1.193420,0.971495,5.374060,33.935900,16.90830,0.324729,9.981450,0.193418,0.028505,0.167589,0.189830,0.098797,AU-Preston,CLMU5
3,LWup,detailed,13.57490,11.714500,1.240660,0.968380,4.305360,40.890700,19.44530,0.369289,11.714500,0.240653,0.031620,0.151606,0.131948,0.094386,AU-Preston,CLMU5
4,Qle,baseline,22.95310,-3.692590,0.966173,0.708790,9.611410,12.137400,36.84560,0.750909,3.692590,0.033828,0.291210,0.209044,0.219815,0.276042,AU-Preston,CLMU5
